There is actually shortcut for what we do yesterday. But what we do yesterday allow for customization

First, create a directory called blog/templates/blog. Then create the following templates in the directory

first blog/templates/blog/blogpost_detail.html

In [ ]:
<html>
    <head><title>List View</title></head>
    <body>
            <p>{{object.title}}</p>
            <p>{{object.content}}</p>
    </body>
</html>


blog/templates/blog/blogpost_form.html

In [ ]:
<html>
    <head><title>Form view</title></head>
    <body>
        <form action="" method="post">{% csrf_token %}
            {{ form.as_p }}
            <input type="submit" value="Save" />
        </form>
    </body>
</html>


blog/templates/blog/blogpost_list.html

In [ ]:
<html>
<head><title>List View</title></head>
<body>
  {% for i in object_list %}
  <p>{{i.title}}</p>
  {% endfor %}
</body>
</html>


You also need to add a method for models before continue

In [ ]:
from django.db import models
from django.urls import reverse

# Create your models here.
class BlogPost(models.Model):
    title = models.CharField(max_length=100)
    content = models.TextField()

    def get_absolute_url(self):
        return reverse("blog-view", kwargs={"pk":self.pk})

    def __str__(self):
        return self.title


Now change all the views in blog/views.py into the following

In [ ]:
from django.views.generic import ListView
from django.views.generic import DetailView
from django.views.generic.edit import CreateView
from django.views.generic.edit import UpdateView
from django.views.generic.edit import DeleteView
from blog.models import BlogPost
from django.urls import reverse_lazy


# Create your views here.
class BlogListView(ListView):
    model = BlogPost


class BlogView(DetailView):
    model = BlogPost

class BlogCreateView(CreateView):
    model = BlogPost
    fields = ["title", "content"]


class BlogEditView(UpdateView):
    model = BlogPost
    fields = ["title", "content"]


class BlogDeleteView(DeleteView):
    model = BlogPost
    success_url = reverse_lazy("blog-list")


This take shorter code, but the previous views can have more logic. Choose one

## Let clean up the templates

So we start with some setup. Ceate a directory for {project_name}/templates. Then edit {project_name}/settings.py

In [ ]:
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [ os.path.join(BASE_DIR, "templates") ],
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        },
    },
]


Essentially change DIRS into that format. Now add a templates called {project_name}/templates/base.html

In [ ]:
<html>
    <head><title>{% block title %}{% endblock %}</title>
    </head>
    <body>
        {% block content %}{% endblock %}
    </body>
</html>


Now change the create form in blog/templates/form/blogpost_form.html

In [ ]:
{% extends "base.html" %}
{% block content %}
        <form action="" method="post">{% csrf_token %}
            {{ form.as_p }}
            <input type="submit" value="Save" />
        </form>
{% endblock %}


Edit the template for detail

In [ ]:
{% extends "base.html" %}
{% block content %}
    <p>{{object.title}}</p>
    <p>{{object.content}}</p>
{% endblock %}


Now change tthe template blog/templates/form/blogpost_list.html

In [ ]:
{% extends "base.html" %}
{% block content %}
{% for i in object_list %}
  <p>{{i.title}}</p>
  {% endfor %}
{% endblock %}


First thing you learn here is template inheritance. Also I will take the opportunity to explain the template engine syntax.

Does {{ }} means it is a value from the views. The dictionary we pass render function. 

In [ ]:
{{value}}


If you want to refer a key in dictionary and a method name/ property you use .

In [ ]:
{{value.something}}

{% %} is for special template function. 

In [ ]:
{% extends %}

Some function need to open and close. for example {% block %} and {% for %}. You just put {% endblock %} or {% endfor %}

Finally I am going to just add a for and show you that you can show value in links. 

In [ ]:
{% extends "base.html" %}
{% block title %}List View{% endblock %}
{% block content %}
{% for i in object_list %}
<p><a href="{{i.get_absolute_url}}">{% if i.title %}{{i.title}}{% else %}No title for {{i.id}}{% endif %}</a></p>
  {% endfor %}
{% endblock %}


I just want to show the if function. Also the fact that you can call i even I is a methosd of a class

Just for kicks, lets add something into the views. Update the import so that it look like the top. And update the BlogListView with get_context_data method like below. 

You can copy the get_context_data and put it in all views is you want to.

In [ ]:
from django.views.generic import ListView
from django.views.generic import DetailView
from django.views.generic.edit import CreateView
from django.views.generic.edit import UpdateView
from django.views.generic.edit import DeleteView
from blog.models import BlogPost
from django.urls import reverse_lazy
import datetime


# Create your views here.
class BlogListView(ListView):
    model = BlogPost

    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        context["now"] = datetime.datetime.now()
        context["misc"] = {"a":1, "b":2}
        return context


Now update templates/base.html into

In [ ]:
<html>
    <head><title>{% block title %}{% endblock %}</title></head>
    {% block extra_head %} {% endblock %}
    <body>
        <p>{{ now }}</p>
        <p>{{ misc.a }}</p>
        {% block content %}{% endblock %}
    </body>
</html>